In [2]:
import pandas as pd

df = pd.read_csv("data/Valiant 2025 - Looker Studio - Scrim champs.csv")

print(df.columns)


Index(['Date', 'Top', 'Jgl', 'Mid', 'Bot', 'Sup', 'Patch', 'Win - Lose',
       'Opponent', 'E-Top', 'E-Jgl', 'E-Mid', 'E-Bot', 'E-Sup', 'side',
       'Winrate since the start', '58,48%',
       'Winrate Red: 057% | Winrate Blue: 062%', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Champion'],
      dtype='object')


### Garder que les bonnes colonnes (erreur dans fichier csv)

In [3]:
colonnes_a_garder = ['Date', 'Top', 'Jgl', 'Mid', 'Bot', 'Sup', 'Patch', 'Win - Lose',
                      'Opponent', 'E-Top', 'E-Jgl', 'E-Mid', 'E-Bot', 'E-Sup', 'side']

df = df[colonnes_a_garder]

print(df.head())


         Date       Top       Jgl     Mid      Bot    Sup  Patch Win - Lose  \
0  03/12/2024       Jax   Skarner    Ahri  Smolder  Rakan  14.23        Win   
1  03/12/2024   Camille  Nocturne   Sylas    Varus   Rell  14.23        Win   
2  03/12/2024    Aurora   Skarner    Yone    Corki   Bard  14.23        Win   
3  03/12/2024  Renekton      Zyra    Yone     Ashe  Leona  14.23        Win   
4  03/12/2024    KSante     Poppy  Aurora  Caitlyn    Lux  14.23        Win   

  Opponent    E-Top       E-Jgl    E-Mid    E-Bot     E-Sup  side  
0      MCK  Ambessa       Viego   Aurora    Corki    Maokai  Blue  
1      MCK  Ambessa    JarvanIV     Ahri     Ashe  Nautilus   Red  
2      MCK    Galio  MonkeyKing   Syndra  Kalista    Renata  Blue  
3      MCK   Aurora     Skarner  Ambessa   Ezreal     Rakan   Red  
4      MCK     Gwen       Viego    Galio  Kalista    Renata  Blue  


In [4]:
df.shape

(301, 15)

# Exploration
### Dataframe pour info de base sur les champions

In [5]:
champion_stats = {'Top': {}, 'Jgl': {}, 'Mid': {}, 'Bot': {}, 'Sup': {}}

for index, row in df.iterrows():
    for role in ['Top', 'Jgl', 'Mid', 'Bot', 'Sup']:
        champion = row[role]
        result = row['Win - Lose']
        
        if champion not in champion_stats[role]:
            champion_stats[role][champion] = {'count': 0, 'wins': 0}
        
        champion_stats[role][champion]['count'] += 1
        if result == 'Win':
            champion_stats[role][champion]['wins'] += 1

winrate_stats = {'Top': [], 'Jgl': [], 'Mid': [], 'Bot': [], 'Sup': []}
total_matches = len(df) 

for role in ['Top', 'Jgl', 'Mid', 'Bot', 'Sup']:
    for champion, stats in champion_stats[role].items():
        winrate = stats['wins'] / stats['count'] if stats['count'] > 0 else 0
        pickrate = stats['count'] / total_matches  
        winrate_stats[role].append({
            'Champion': champion,
            'winrate': round(winrate, 2),
            'pickrate': round(pickrate, 2),
            'count': stats['count'],
            'role': role  
        })

dfs = {}
for role, champions in winrate_stats.items():
    df_role = pd.DataFrame(champions)
    df_role['role'] = role  
    dfs[role] = df_role.nlargest(5, 'count')  

final_df = pd.concat(dfs.values(), ignore_index=True)

final_df = final_df.dropna(subset=['Champion'])

print(final_df)

    Champion  winrate  pickrate  count role
1     KSante     0.47      0.11     34  Top
2    Ambessa     0.62      0.10     29  Top
3      Jayce     0.55      0.07     22  Top
4   Renekton     0.71      0.07     21  Top
6     Maokai     0.69      0.10     29  Jgl
7         Vi     0.71      0.08     24  Jgl
8      Ivern     0.43      0.07     21  Jgl
9       Zyra     0.58      0.06     19  Jgl
11      Ahri     0.72      0.08     25  Mid
12      Yone     0.43      0.07     21  Mid
13    Viktor     0.60      0.07     20  Mid
14   Orianna     0.58      0.06     19  Mid
16     Varus     0.57      0.15     46  Bot
17      Ashe     0.59      0.10     29  Bot
18     Corki     0.67      0.09     27  Bot
19     Kaisa     0.55      0.07     20  Bot
21      Rell     0.58      0.15     45  Sup
22     Leona     0.65      0.13     40  Sup
23  Nautilus     0.54      0.08     24  Sup
24     Rakan     0.60      0.07     20  Sup


### Top 5 champions winrate aux differents roles

In [6]:
top_performers = {}

for role in ['Top', 'Jgl', 'Mid', 'Bot', 'Sup']:
    role_stats = df.groupby(role)['Win - Lose'].value_counts().unstack(fill_value=0)
    
    role_stats['winrate'] = role_stats['Win'] / (role_stats['Win'] + role_stats['Lose'])
    
    role_stats = role_stats[role_stats['Win'] + role_stats['Lose'] >= 5] # champion avec minimum 5 patch
    
    top_performers[role] = role_stats.sort_values('winrate', ascending=False).head(5)

for role, champions in top_performers.items():
    print(f"Top 5 champions winrate aux differents roles:\n", champions[['winrate']], "\n")


Top 5 champions winrate aux differents roles:
 Win - Lose   winrate
Top                 
Jax         0.923077
Ornn        0.888889
Camille     0.818182
Volibear    0.800000
Gnar        0.750000 

Top 5 champions winrate aux differents roles:
 Win - Lose   winrate
Jgl                 
JarvanIV    0.857143
Skarner     0.818182
Viego       0.733333
Vi          0.708333
Maokai      0.689655 

Top 5 champions winrate aux differents roles:
 Win - Lose   winrate
Mid                 
Syndra      0.833333
Smolder     0.800000
Ahri        0.720000
Taliyah     0.714286
Aurora      0.700000 

Top 5 champions winrate aux differents roles:
 Win - Lose    winrate
Bot                  
Caitlyn      0.777778
MissFortune  0.777778
Kalista      0.700000
Ezreal       0.666667
Corki        0.666667 

Top 5 champions winrate aux differents roles:
 Win - Lose   winrate
Sup                 
Blitzcrank  0.800000
Poppy       0.727273
Thresh      0.666667
Leona       0.650000
Renata      0.625000 



# Test Machine Learning

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier


df = df.dropna(subset=['Top', 'Jgl', 'Mid', 'Bot', 'Sup', 'Win - Lose'])

# Convertit win en valeur 1 = objectif et lose en 0
df['Win - Lose'] = df['Win - Lose'].apply(lambda x: 1 if x == 'Win' else 0)

# Encode les valeurs numériques
le = LabelEncoder()

df['Top'] = le.fit_transform(df['Top'])
df['Jgl'] = le.fit_transform(df['Jgl'])
df['Mid'] = le.fit_transform(df['Mid'])
df['Bot'] = le.fit_transform(df['Bot'])
df['Sup'] = le.fit_transform(df['Sup'])

df['Opponent'] = le.fit_transform(df['Opponent'])
df['side'] = le.fit_transform(df['side'])

#convertir chaine de carac en nombre
df['Patch'] = df['Patch'].str.replace(r'[^\d.]+', '', regex=True)

# Convertir en float
df['Patch'] = pd.to_numeric(df['Patch'], errors='coerce')

#print(df.head())

X = df[['Top', 'Jgl', 'Mid', 'Bot', 'Sup', 'Patch', 'Opponent', 'side']]  
y = df['Win - Lose']





In [10]:
X.shape

(213, 8)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42,stratify=y)

model = RandomForestClassifier(n_estimators=250, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

#print("Classification Report:")
#print(classification_report(y_test, y_pred))

#interpretation des résultats
importances = model.feature_importances_
features = X.columns

importance_df = pd.DataFrame({'Feature': features, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print(importance_df)

Accuracy: 0.50
    Feature  Importance
6  Opponent    0.207111
2       Mid    0.140568
1       Jgl    0.136917
4       Sup    0.128286
0       Top    0.128106
3       Bot    0.120339
5     Patch    0.089223
7      side    0.049451


# StreamLit

In [14]:
import streamlit as st

code =""
# Streamlit Title
st.title("Top 5 Champions by Role")

# Sélection du rôle via un menu déroulant
role = st.selectbox("Select a Role", ['Top', 'Jgl', 'Mid', 'Bot', 'Sup'])

# Filtrer les 5 champions les plus joués dans le rôle sélectionné
role_data = final_df[final_df['role'] == role].nlargest(5, 'count')

# Affichage des 5 champions dans le rôle
st.write(f"Top 5 champions in {role} role:")
st.dataframe(role_data[['Champion', 'winrate', 'pickrate', 'count']])

# Sauvegarder ce code dans un fichier .py
with open("streamlit_app.py", "w") as f:
    f.write(code)

print("Le fichier streamlit_app.py a été créé.")

2025-03-11 19:53:36.003 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-11 19:53:36.220 
  command:

    streamlit run C:\Users\33631\AppData\Roaming\Python\Python313\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-03-11 19:53:36.221 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-11 19:53:36.221 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-11 19:53:36.221 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-11 19:53:36.222 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-11 19:53:36.223 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-11 19:53:36.224 Sess

Le fichier streamlit_app.py a été créé.
